<a href="https://colab.research.google.com/github/mzohaibnasir/cLabs_Miscellaneous/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## installing required packages

In [ ]:

!pip install langchain_openai
!pip install langchain_core
!pip install python-dotenv
!pip install langchain_community
!pip install langserve
!pip install bs4
!pip install pypdf
!pip install chromadb
# faiss-cpu #Faiss is a library — developed by Facebook AI — that enables efficient similarity search. So, given a set of vectors, we can index them using Faiss — then using another vector (the query vector), we search for the most similar vectors within the index.
!pip install pinecone
!pip install langchainhub
# !pip install groq
!pip install langchain-groq
# sentence_transformers

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate


from langchain.chains import RetrievalQA

from langchain.chains import create_retrieval_chain

from dotenv import load_dotenv

load_dotenv()

## Preparing data

In [2]:
# Mount your Google Drive to the Colab environment
import os

if not os.path.exists("PDFs"):
    os.makedirs("PDFs")


from google.colab import drive
drive.mount('/content/gdrive')

# !ls /content/gdrive/MyDrive
! cp /content/gdrive/MyDrive/crime-and-punishment.pdf /content/PDFs/



Mounted at /content/gdrive


In [56]:
from langchain_community.document_loaders import PyPDFLoader


try:
  loader = PyPDFLoader("./PDFs/crime-and-punishment.pdf")
  pages = loader.load()
  print(f"Total Docs: {len(pages)}")
except Exception as e:
  print(f"Error loading PDF: {e}")

pages[3].page_content

Total Docs: 767


'Crime and Punishment \x18epilepsy, from which he suffered for the rest of his life. The \nfits occurred three or four times a year and were more fre -\nquent in periods of great strain. In 1859 he was allowed to \nreturn to Russia. He started a journal— ‘Vremya,’ which was \nforbidden by the Censorship through a misunderstanding. \nIn 1864 he lost his first wife and his brother Mihail. He was \nin terrible poverty, yet he took upon himself the payment \nof his brother’s debts. He started another journal—‘The Ep -\noch,’ which within a few months was also prohibited. He \nwas weighed down by debt, his brother’s family was depen -\ndent on him, he was forced to write at heart-breaking speed, \nand is said never to have corrected his work. The later years \nof his life were much softened by the tenderness and devo -\ntion of his second wife.\nIn June 1880 he made his famous speech at the unveil -\ning of the monument to Pushkin in Moscow and he was \nreceived with extraordinary demonstra

## Data Preprocessing

### Combining pages and replace tabs with spaces

In [77]:
# Combine the pages, and replace the tabs with spaces
content = pages[6:]
corpus = ' '.join([page.page_content.replace('\t', ' ') for page in content])
print(f"length of Corpus: {len(corpus)}, \n\n\ncorpus[:100]: {corpus[:100]}")

length of Corpus: 1167908, 


corpus[:100]:  Free eBooks at Planet eBook.comChapter I
On an exceptionally hot evening early in July a young 
ma


In [78]:
print(f"length of Corpus: {len(corpus)}, \n\n\ncorpus[:100]: {corpus[:100]}")


length of Corpus: 1167908, 


corpus[:100]:  Free eBooks at Planet eBook.comChapter I
On an exceptionally hot evening early in July a young 
ma


In [94]:
import re
def clean_corpus(text):
   # Remove the specific phrase 'Free eBooks at Planet eBook.com' and surrounding whitespace
   cleaned_text = re.sub(r'\s*Crime and Punishment\s*', '', text)
   cleaned_text = re.sub(r'Free eBooks at Planet eBook\.com\s*', '', cleaned_text)

   # Remove extra spaces
   cleaned_text = re.sub(r' +', ' ', cleaned_text)
  #  Remove non-printable characters
   cleaned_text = re.sub(r'[\x00-\x1F]', ' ', cleaned_text)
   # Replace newline characters with spaces
   cleaned_text = cleaned_text.replace('\n', ' ') # replace newline with space
   cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text) # remove spaces around '-'
   return cleaned_text


cleaned_corpus =clean_corpus(corpus)
print(f"length of cleaned_corpus: {len(cleaned_corpus)}, \n\n\ncleaned_corpus[:100]: {cleaned_corpus[:500]}")

length of cleaned_corpus: 1137847, 


cleaned_corpus[:100]:   Chapter I On an exceptionally hot evening early in July a young  man came out of the garret in which he lodged in S.  Place and walked slowly, as though in hesitation, towards  K. bridge. He had successfully avoided meeting his landlady on the  staircase. His garret was under the roof of a high, fivestoried house and was more like a cupboard than a room. The  landlady who provided him with garret, dinners, and attendance, lived on the floor below, and every time he went  out he was obliged to 


## LLM

In [11]:
from google.colab import userdata

OPENAIAPIKEY = userdata.get("OPENAI_API_KEY")


1. Langchain

In [97]:
from langchain_openai import OpenAI
openAIclient = OpenAI(api_key=OPENAIAPIKEY ,
                      temperature=0,
                      max_tokens=1000,
                      model='gpt-3.5-turbo')


# lets see how many tokens we have in the book
openAIclient.get_num_tokens(corpus), openAIclient.get_num_tokens(cleaned_corpus)



(301799, 284490)



*   Total no. of Tokens in book: **301799**
*   No. of tokens after Preprocessing : **284490**
*   Total Margin : **17309**

2. HFHub

3. OLlama

In [ ]:
! pip install ollama
! ollama run llama3

In [103]:
from langchain_community.llms import  Ollama
from langchain_core.prompts import ChatPromptTemplate  # usefull for chatbot
from langchain_core.output_parsers import (
    StrOutputParser,
)  # You can also create a custom output parser.

# openAI: llama2
ollamaclient = Ollama(model="llama2")
# output_parser = StrOutputParser()  # responsible for getting output

# prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant. Please respond to queries"),
        ("user", "Question:{question}"),
    ]
)

output_parser = StrOutputParser()  # responsible for getting output
chain = prompt | ollamaclient | output_parser
chain.invoke({"question": "What is this about?"})

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7e4f07f78a00>: Failed to establish a new connection: [Errno 111] Connection refused'))